In [1]:
import pandas as pd

from risk_data import get_factor_data, get_factor_master
from risk_chart import px_write

from risk_corr_mds import *

In [ ]:
from importlib import reload
import risk_lib.corr_mds
reload(risk_lib.corr_mds)
from risk_corr_mds import *

In [ ]:
# TODO: Try diagonal and uniform correlations (totally uncorrelated and correlated) (think about the circle)
# TODO: Make sure we're finding global minimum
# TODO: With and without portfolio
# TODO: Paramters - factors, dates, correlation
# TODO: Remove first principal component... make it beta neutral

In [ ]:
halflifes = [21, 63, 126, 252]
factor_data = get_factor_data(factor_set='read') #, check=is_data_stale)
build_factor_data

In [5]:
factor_master = get_factor_master()
factor_master.query('source==composite').index.empty

True

In [ ]:
# TODO: Do the args work?!

args = {'random_state': 42, 
        'n_init': 100}

dates_t1 = ['2024-11-20', '2024-10-18', '2024-08-16', '2024-01-05']
dates_t1 = ['2024-12-11', '2024-10-18', '2024-08-16', '2024-01-05']
dates_t0 = ['2024-10-31', '2024-09-30', '2024-08-30']

# TCW as of 2024-10-31
# TCW + 3 themes as of 2024-10-31
# TCW + 3 themes + trump as of 2024-11-18

fig = (run_mds(factor_data, 
               transformation='rotate_initial', 
               dates=dates_t0,
               start_date='2024', 
               tick_range=1,
               drop_composites=True,
               drop_trump=True,
               **args)
    #    .update_layout(title=f"random_state={args['random_state']}, n_init={args['n_init']}")
       )
fig.show(renderer='png')
# fig.write_image('corr_tcw.png')
px_write(fig, 'corr_tcw.png')


fig = (run_mds(factor_data, 
               transformation='rotate_initial', 
               dates=dates_t0,
               start_date='2024', 
               tick_range=1,
               drop_composites=False,
               drop_trump=True,
               **args)
    #    .update_layout(title=f"random_state={args['random_state']}, n_init={args['n_init']}")
       )
fig.show(renderer='png')
# fig.write_image('corr_theme.png')
px_write(fig, 'corr_theme.png')

fig = (run_mds(factor_data, 
               transformation='rotate_initial', 
               dates=dates_t1,
               start_date='2024', 
               tick_range=1,
               drop_composites=False,
               drop_trump=False,
               **args)
    #    .update_layout(title=f"random_state={args['random_state']}, n_init={args['n_init']}")
       )
fig.show(renderer='png')
# fig.write_image('corr_trump.png')
px_write(fig, 'corr_trump.png')




In [ ]:

# TODO: Do the args work?!

args = {'random_state': 42, 
        'n_init': 100}

dates_t1 = ['2024-11-20', '2024-10-18', '2024-08-16', '2024-01-05']
dates_t1 = ['2024-12-11', '2024-10-18', '2024-08-16', '2024-01-05']
dates_t0 = ['2024-10-31', '2024-09-30', '2024-08-30']

# TCW as of 2024-10-31
# TCW + 3 themes as of 2024-10-31
# TCW + 3 themes + trump as of 2024-11-18

fig = (run_mds(factor_data, 
               transformation='rotate_initial', 
               dates=dates_t0,
               start_date='2024', 
               tick_range=1,
               drop_composites=True,
               drop_trump=True,
               **args)
    #    .update_layout(title=f"random_state={args['random_state']}, n_init={args['n_init']}")
       )
fig.show(renderer='png')
# fig.write_image('corr_tcw.png')
px_write(fig, 'corr_tcw.png')


fig = (run_mds(factor_data, 
               transformation='rotate_initial', 
               dates=dates_t0,
               start_date='2024', 
               tick_range=1,
               drop_composites=False,
               drop_trump=True,
               **args)
    #    .update_layout(title=f"random_state={args['random_state']}, n_init={args['n_init']}")
       )
fig.show(renderer='png')
# fig.write_image('corr_theme.png')
px_write(fig, 'corr_theme.png')

fig = (run_mds(factor_data, 
               transformation='rotate_initial', 
               dates=dates_t1,
               start_date='2024', 
               tick_range=1,
               drop_composites=False,
               drop_trump=False,
               **args)
    #    .update_layout(title=f"random_state={args['random_state']}, n_init={args['n_init']}")
       )
fig.show(renderer='png')
# fig.write_image('corr_trump.png')
px_write(fig, 'corr_trump.png')




In [ ]:
factor_data.date.sel(date=slice('2020', None)).values

In [ ]:
args = {'random_state': 42, 
        'n_init': 100}

# dates_t1 = ['2024-11-20', '2024-10-18', '2024-08-16']
# dates_t0 = ['2024-10-31', '2024-09-30', '2024-08-30']

dates_t1 = ['2024-11-20', '2024-10-18', '2024-08-16', '2024-01-05']
dates_t1 = ['2024-12-11', '2024-10-18', '2024-08-16', '2024-01-05']
dates_t0 = ['2024-10-31', '2024-09-30', '2024-08-30']
dates = factor_data.date.sel(date=slice('2020', None)).values


fig = (run_mds(factor_data.resample(date='W').last(), 
               transformation='rotate_initial', 
               dates=dates,
               start_date='2020', 
               tick_range=1,
               animate=True,
               drop_composites=True,
               drop_trump=False,
               **args)
    #    .update_layout(title=f"random_state={args['random_state']}, n_init={args['n_init']}")
       )
fig.show()
# fig.write_html('corr_animation.html')

In [ ]:
# reload(corr_mds)
# run_mds(factor_data.corr, transformation=None, start_date='2004', tick_range='auto').show(renderer='png')
# run_mds(factor_data.corr, transformation='rotate', start_date='2004', tick_range='auto').show(renderer='png')
# run_mds(factor_data.corr, transformation='rotate_initial', start_date='2004', tick_range='auto').show(renderer='png')
# # run_mds(factor_data.corr, transformation='normalize', start_date='2004', tick_range='auto').show(renderer='png')
# # run_mds(factor_data.corr, transformation='rotate_list', start_date='2004', tick_range='auto').show(renderer='png')
# # run_mds(factor_data.corr, transformation='normalize', start_date='2004', tick_range=1).show(renderer='png')
# # fig.show(renderer='png')
